In [10]:
def format_number(num):
    if num >= 1e12:
        return f"{num/1e12:.0f} T"
    elif num >= 1e9:  # Check if the number is in billions
        return f"{num/1e9:.0f} B"
    elif num >= 1e6:  # Check if the number is in millions
        return f"{num/1e6:.0f} M"
    elif num >= 1e3:  # Check if the number is in thousands
        return f"{num/1e3:.0f} K"
    else:  # If the number is less than 1000
        return str(num)

In [12]:
print(format_number(999))   # Output: 12K
print(format_number(365123456))  # Output: 365M
print(format_number(3651234567891011))  # Output: 365M


999
365 M
3651 T


In [2]:
import os
import torch
from torchvision import datasets, transforms

def load_dataset_with_classes_to_drop(root_dir, classes_to_drop):
    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    # Load the dataset
    dataset = datasets.ImageFolder(root=root_dir, transform=transform)

    # Filter out samples of classes to drop
    dataset.samples = [(image, label) for image, label in dataset.samples 
                       if dataset.classes[label] not in classes_to_drop]

    # Update class_to_idx and classes to reflect the removal of classes
    dataset.class_to_idx = {cls_name: idx for cls_name, idx in dataset.class_to_idx.items() 
                            if cls_name not in classes_to_drop}
    dataset.classes = list(dataset.class_to_idx.keys())

    # Update the labels in samples to ensure they match the new class_to_idx
    dataset.samples = [(image, dataset.class_to_idx[dataset.classes[label]]) for image, label in dataset.samples]

    return dataset

# Example usage
root_dir = 'D:/Casper/Data/Animals-10/raw-img'
# selected_folders = ['folder1', 'folder2']  # List the folders you want to include
classes_to_drop = ['cane', 'gatto']  # List the classes you want to drop
dataset = load_dataset_with_classes_to_drop(root_dir, classes_to_drop)



IndexError: list index out of range